## RGF experiment
- It is said to be more powerful than catboost by some Kaggler. So I had a try.  
- The training takes 3+ hours so it's costly.  
- The result is nearly the best, but not a leap in performance. So considering the time consumption, we discard this model.

In [1]:
from rgf.sklearn import RGFRegressor
import numpy as np
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from rgf.sklearn import FastRGFRegressor, RGFRegressor

In [ ]:
x_train = pd.read_csv('../data/subtrain/train_1.tsv', sep='\t')
x_valid = pd.read_csv('../data/subtrain/valid_1.tsv', sep='\t')

In [ ]:
y_train = x_train.target
x_train = x_train.drop(['record_number', 'target'], axis=1)
y_valid = x_valid.target
x_valid = x_valid.drop(['record_number', 'target'], axis=1)

In [ ]:
x_train[:5]
x_train.shape
x_valid.shape
y_train.shape
y_valid.shape

In [ ]:
regressor = RGFRegressor(verbose=True)

In [ ]:
history = regressor.fit(x_train, y_train)

In [ ]:
import pickle

pickle.dump(regressor, open( "../para/rgf.p", "wb" ) )

In [ ]:
load_file = pickle.load( open( "../para/rgf.p", "rb" ) )
load_file.predict(x_valid)

In [ ]:
y_pred = regressor.predict(x_valid)

In [ ]:
y_pred.shape

In [ ]:
preds = np.round(y_pred)
residual = (y_valid - preds).astype("int")
loss = np.where(residual < 0, residual * -0.6, residual * 0.4) 
np.mean(loss)

In [ ]:
quiz_set = pd.read_csv('../data/parsed_quiz.tsv', sep='\t')

In [ ]:
quiz_set.shape
quiz_set.columns

In [ ]:
x_valid.shape
x_valid.columns

In [ ]:
record_number = quiz_set['record_number']
quiz_pred_set = quiz_set.drop(['record_number'], axis = 1)
record_number
quiz_pred_set.shape

In [ ]:
quiz_pred = regressor.predict(quiz_pred_set)

In [ ]:
real_quiz_set = pd.read_csv('../data/quiz.tsv', sep='\t')

In [ ]:
real_quiz_set['target'] = pd.Series(np.round(quiz_pred))

In [6]:
from datetime import datetime, timedelta
def add_func(row):
    acct = row['acceptance_scan_timestamp']
    dd = row['target']
    cdate = datetime.strptime(acct.split()[0], "%Y-%m-%d")
    cdate = cdate + timedelta(days=dd)
    return cdate.strftime("%Y-%m-%d")

In [ ]:
res_set = real_quiz_set[['record_number', 'acceptance_scan_timestamp', 'target']]
res_set['arrive_date'] = res_set.apply(add_func, axis=1)
print("null res:", sum(res_set['arrive_date'].isnull()))

In [ ]:
res_set.drop(columns=['acceptance_scan_timestamp', 'target']).to_csv('../result/rgf_result.tsv', header=None, index=None, sep='\t')

### Some Tuning

In [ ]:
train = pd.read_csv('../data/subtrain/train_1.tsv', sep='\t')
valid = pd.read_csv('../data/subtrain/valid_1.tsv', sep='\t')

In [ ]:
s1_train = train.sample(frac = 0.1, random_state = 1)
s1_valid = valid.sample(frac = 0.1, random_state = 1)
y1_train = s1_train.target
x1_train = s1_train.drop(['record_number', 'target'], axis=1)
y1_valid = s1_valid.target
x1_valid = s1_valid.drop(['record_number', 'target'], axis=1)

In [ ]:
regressor1 = RGFRegressor(verbose=True)
history = regressor1.fit(x1_train, y1_train)

In [ ]:
y1_pred = regressor1.predict(x1_valid)
preds = np.round(y1_pred)
residual = (y1_valid - preds).astype("int")
loss = np.where(residual < 0, residual * -0.6, residual * 0.4) 
np.mean(loss)

In [ ]:
regressor2 = RGFRegressor(verbose=True, l2=0.01)
history = regressor2.fit(x1_train, y1_train)

In [ ]:
y2_pred = regressor2.predict(x1_valid)
preds = np.round(y2_pred)
residual = (y1_valid - preds).astype("int")
loss = np.where(residual < 0, residual * -0.6, residual * 0.4) 
np.mean(loss)

In [ ]:
regressor3 = RGFRegressor(l2=0.001)
regressor3.fit(x1_train, y1_train)
y3_pred = regressor3.predict(x1_valid)
preds = np.round(y3_pred)
residual = (y1_valid - preds).astype("int")
loss = np.where(residual < 0, residual * -0.6, residual * 0.4) 
np.mean(loss)

## Run with adjusted l2

In [2]:
x_train = pd.read_csv('../data/subtrain/train_1.tsv', sep='\t')
x_valid = pd.read_csv('../data/subtrain/valid_1.tsv', sep='\t')
y_train = x_train.target
x_train = x_train.drop(['record_number', 'target'], axis=1)
y_valid = x_valid.target
x_valid = x_valid.drop(['record_number', 'target'], axis=1)

In [3]:
regressor = RGFRegressor(verbose=True, l2=0.001)
history = regressor.fit(x_train, y_train)

/home/zhangr235_gmail_com/anaconda3/envs/py38/lib/python3.8/site-packages/rgf/utils.py:224: UserWarning: Cannot find FastRGF executable files. FastRGF estimators will be unavailable for usage.
  warnings.warn("Cannot find FastRGF executable files. "


"train": 
   algorithm=RGF
   train_x_fn=/tmp/rgf/1d667160-7cb0-42a8-bc58-2b0be78271851.train.data.x
   train_y_fn=/tmp/rgf/1d667160-7cb0-42a8-bc58-2b0be78271851.train.data.y
   Log:ON
   model_fn_prefix=/tmp/rgf/1d667160-7cb0-42a8-bc58-2b0be78271851.model
--------------------
Sat Jan  1 22:14:47 2022: Reading training data ... 
Sat Jan  1 22:15:07 2022: Start ... #train=5889444
--------------------
Forest-level: 
   loss=LS
   max_leaf_forest=500
   max_tree=250
   opt_interval=100
   test_interval=100
   num_tree_search=1
   Verbose:ON
   memory_policy=Generous
-------------
Training data: 17x5889444, nonzero_ratio=0.7941; managed as dense data.
-------------
Optimization: 
   loss=LS
   num_iteration_opt=10
   reg_L2=0.001
   opt_stepsize=0.5
   NormalizeTarget:ON
Tree-level: min_pop=10
Node split: reg_L2=0.001
--------------------
Sat Jan  1 22:21:18 2022: Calling optimizer with 21 trees and 100 leaves
Sat Jan  1 22:21:53 2022: Writing model: seq#=1
Sat Jan  1 22:27:21 2022: Callin

In [4]:
quiz_set = pd.read_csv('../data/parsed_quiz.tsv', sep='\t')
quiz_pred_set = quiz_set.drop(['record_number'], axis = 1)
quiz_pred = regressor.predict(quiz_pred_set)

"predict": 
   model_fn=/tmp/rgf/1d667160-7cb0-42a8-bc58-2b0be78271851.model-05
   test_x_fn=/tmp/rgf/1d667160-7cb0-42a8-bc58-2b0be78271851.test.data.x
   prediction_fn=/tmp/rgf/1d667160-7cb0-42a8-bc58-2b0be78271851.predictions.txt
   Log:ON
--------------------
Sat Jan  1 23:33:34 2022: Reading test data ... 
Sat Jan  1 23:33:40 2022: Predicting ... 
elapsed: 6.11598
/tmp/rgf/1d667160-7cb0-42a8-bc58-2b0be78271851.predictions.txt: /tmp/rgf/1d667160-7cb0-42a8-bc58-2b0be78271851.model-05,#leaf=501,#tree=60
Sat Jan  1 23:35:08 2022: Done ... 



In [7]:
real_quiz_set = pd.read_csv('../data/quiz.tsv', sep='\t')
real_quiz_set['target'] = pd.Series(np.round(quiz_pred))
res_set = real_quiz_set[['record_number', 'acceptance_scan_timestamp', 'target']]
res_set['arrive_date'] = res_set.apply(add_func, axis=1)
print("null res:", sum(res_set['arrive_date'].isnull()))
res_set.drop(columns=['acceptance_scan_timestamp', 'target']).to_csv('../result/rgf2_result.tsv', header=None, index=None, sep='\t')

/tmp/ipykernel_11951/2922784003.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  res_set['arrive_date'] = res_set.apply(add_func, axis=1)


null res: 0
